In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd 
data_train = pd.read_csv('/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Tamil/tamil_sentiment_full_train.tsv', sep = '\t')
data_dev = pd.read_csv('/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Tamil/tamil_sentiment_full_dev.tsv', sep = '\t')
data_test = pd.read_csv('/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Tamil/tamil_sentiment_full_test_withoutlabels.tsv', sep = '\t')

In [3]:
data_train['category'].value_counts()

Positive          20069
unknown_state      5628
Negative           4271
Mixed_feelings     4020
not-Tamil          1667
Positive              1
Name: category, dtype: int64

In [4]:
data_train['category'] = data_train['category'].replace('Positive ', 'Positive')

In [5]:
data_train['category'].value_counts()

Positive          20070
unknown_state      5628
Negative           4271
Mixed_feelings     4020
not-Tamil          1667
Name: category, dtype: int64

In [6]:
import numpy as np
Samplesize = 1667  #number of samples that you want       
df_data_train = data_train.groupby('category', as_index=False).apply(lambda array: array.loc[np.random.choice(array.index, Samplesize, True),:])


In [7]:
df_data_train['category'].value_counts()

Mixed_feelings    1667
unknown_state     1667
Negative          1667
not-Tamil         1667
Positive          1667
Name: category, dtype: int64

In [8]:
df_data_train

text        category
0 18292  போய் பொளப்ப பாருங்க நண்பர்களே     நாம்ம உழைச்ச...  Mixed_feelings
  19267  Trailer ennamo nalla dhan iruku.....election m...  Mixed_feelings
  9579       2019 Diwali treat  Bigil - like  Kaithi - cmt  Mixed_feelings
  35598  Ellarum surya vaa troll panuranga enna 34 movi...  Mixed_feelings
  20475  20k dislikes....ha ha ha ha.....u cant stop ou...  Mixed_feelings
...                                                    ...             ...
4 31992          Any Malayalies   Rajani Sir Hit like Here   unknown_state
  31225  Trailer semma and padam Vera level la irukum  ...   unknown_state
  6140                        Hip Hop tamizha FANS unda 🤣🤣   unknown_state
  2352                     Only for sk anna fans like here   unknown_state
  23883             100k likes within 30minutes thala mass   unknown_state

[8335 rows x 2 columns]

In [9]:
from sklearn import preprocessing
import numpy as np

le = preprocessing.LabelEncoder()

le.fit(df_data_train['category'])
print(le.classes_)

y_train_index = le.transform(df_data_train['category'])
y_dev_index = le.transform(data_dev['category'])

['Mixed_feelings' 'Negative' 'Positive' 'not-Tamil' 'unknown_state']


In [10]:
from keras.utils.np_utils import to_categorical
y_train_cal  =  to_categorical(y_train_index)
y_val_cal  =  to_categorical(y_dev_index)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvec = TfidfVectorizer(ngram_range=(1,6), analyzer = 'char_wb', max_features = 30000)
#tfidfvec = TfidfVectorizer(ngram_range=(1,3), max_features = 30000)
tfidf_train = tfidfvec.fit_transform(df_data_train['text'])
tfidf_val = tfidfvec.transform(data_dev['text'])

In [13]:
x_train = tfidf_train
x_test = tfidf_val
y_train = y_train_cal
y_test =y_val_cal

In [14]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(4096, input_dim = tfidf_train.shape[1], activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              122884096 
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2097664   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 125,014,917
Trainable params: 125,014,917
Non-trainable params: 0
__________________________________________

In [15]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train.toarray(), y_train, validation_data=(x_test.toarray(), y_test), epochs=20, batch_size=20, verbose=2 )

Epoch 1/20
417/417 - 292s - loss: 0.3853 - accuracy: 0.5172 - val_loss: 0.3829 - val_accuracy: 0.5255
Epoch 2/20


In [ ]:
tfidf_test = tfidfvec.transform(data_test['text'])

In [ ]:
predictions_test_submission = model.predict(tfidf_test.toarray())

In [ ]:
predictions_test_submission

array([[6.27814301e-10, 3.96231981e-09, 9.99997377e-01, 6.85449706e-08,
        2.47665776e-06],
       [8.34990176e-04, 6.85240608e-04, 7.79213727e-01, 1.19174086e-01,
        1.00092001e-01],
       [1.33145898e-06, 3.01309311e-09, 4.04716593e-05, 4.29477723e-06,
        9.99953866e-01],
       ...,
       [5.43304985e-16, 8.00601827e-13, 1.00000000e+00, 3.94089032e-11,
        5.44785788e-12],
       [9.99830365e-01, 5.28267208e-07, 2.01034745e-05, 3.82885560e-07,
        1.48651539e-04],
       [7.12246845e-07, 2.51298671e-09, 9.99998212e-01, 1.85177498e-07,
        8.76227261e-07]], dtype=float32)

In [ ]:
import numpy as np
predictions_test_submission = np.asarray(predictions_test_submission)

In [ ]:
import numpy as np
predictions_test_submission1 = np.zeros_like(predictions_test_submission)
predictions_test_submission1[np.arange(len(predictions_test_submission)), predictions_test_submission.argmax(1)] = 1

In [ ]:
predictions_test_submission1

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
predictions_test_submission1_final = np.argmax(predictions_test_submission1, axis=1)

In [ ]:
predictions_test_submission1_final1 = le.inverse_transform(predictions_test_submission1_final)

In [ ]:
predictions_test_submission1_final1

array(['Positive', 'Positive', 'unknown state', 'not-Kannada', 'Positive',
       'unknown state', 'Positive', 'not-Kannada', 'Positive', 'Positive',
       'Negative', 'Positive', 'not-Kannada', 'Mixed feelings',
       'Positive', 'Positive', 'Positive', 'Positive', 'Negative',
       'Negative', 'Positive', 'Positive', 'Positive', 'Positive',
       'not-Kannada', 'unknown state', 'not-Kannada', 'Positive',
       'Mixed feelings', 'Negative', 'Mixed feelings', 'not-Kannada',
       'not-Kannada', 'unknown state', 'Negative', 'unknown state',
       'unknown state', 'Positive', 'Positive', 'not-Kannada',
       'Mixed feelings', 'unknown state', 'Positive', 'Positive',
       'unknown state', 'not-Kannada', 'unknown state', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'unknown state',
       'unknown state', 'unknown state', 'Positive', 'Positive',
       'Positive', 'Positive', 'not-Kannada', 'Positive', 'Positive',
       'Mixed feelings', 'Positive', 'Mixed 

In [ ]:
data_test['category'] = predictions_test_submission1_final1

In [ ]:
data_test

,id,text,category
0,Kan_1,ಈ ಹಾಡನ್ನು ಹಾಡಿದ ವಿಜಯ ಪ್ರಕಾಶ voice ಯಾರಿಗೆಲ್ಲಾ ಇ...,Positive
1,Kan_2,Jai D Boss,Positive
2,Kan_3,Signature move,unknown state
3,Kan_4,Super song bro,not-Kannada
4,Kan_5,Wow Super agi helidira sir,Positive
...,...,...,...
763,Kan_764,Thu thukali trailer,Negative
764,Kan_765,Siri gannadam galge haakbitallapa Thu yaro ni...,Negative
765,Kan_766,ನಾವೇನಾದರು ಸ್ವಾಬಿಮಾನಿಯಾಗಿ ಬದುಕನ್ನು ಕಟ್ಟಿಕೊಳ್ಳಲು...,Positive
766,Kan_767,ದಿಯಾ ನೀವು ಸೋಷಿಯಲ್ ಮೀಡಿಯಾ ಅಲ್ಲಿ ಕಾಣ್ತಿಲ್ಲ.nದಯವಿ...,Mixed feelings


In [ ]:
data_test.to_csv("/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Kannada/DNN_Kanada_64.tsv", sep="\t", index=False)

In [ ]:
check_submission = pd.read_csv("/content/drive/MyDrive/HASOC_2021/Sentiment_Dravidian/Kannada/DNN_Kanada_64.tsv", sep="\t")

In [ ]:
check_submission

,id,text,category
0,Kan_1,ಈ ಹಾಡನ್ನು ಹಾಡಿದ ವಿಜಯ ಪ್ರಕಾಶ voice ಯಾರಿಗೆಲ್ಲಾ ಇ...,Positive
1,Kan_2,Jai D Boss,Positive
2,Kan_3,Signature move,unknown state
3,Kan_4,Super song bro,not-Kannada
4,Kan_5,Wow Super agi helidira sir,Positive
...,...,...,...
763,Kan_764,Thu thukali trailer,Negative
764,Kan_765,Siri gannadam galge haakbitallapa Thu yaro ni...,Negative
765,Kan_766,ನಾವೇನಾದರು ಸ್ವಾಬಿಮಾನಿಯಾಗಿ ಬದುಕನ್ನು ಕಟ್ಟಿಕೊಳ್ಳಲು...,Positive
766,Kan_767,ದಿಯಾ ನೀವು ಸೋಷಿಯಲ್ ಮೀಡಿಯಾ ಅಲ್ಲಿ ಕಾಣ್ತಿಲ್ಲ.nದಯವಿ...,Mixed feelings
